In [1]:
import h5py
import numpy as np
import tensorflow as tf
import pickle
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import clone_model
import pathlib
from tqdm import tqdm
import sys
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
attack_vector='network'
machine_setting='dynamic_browser_version' # static_browser_version

root_path=f'/content/drive/MyDrive/Colab Notebooks/concept-drift-online/'
data_path=f'{root_path}/data/{machine_setting}/{attack_vector}'
model_0_train_path=f'2021_08_27/train.h5'


In [11]:
os.getcwd()

'/content'

In [16]:
sys.path.append(str(root_path)) # Add the directory to the path
from utils import prepare_hdf5_data,get_data_paths_ordered,standartize

In [13]:

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [14]:
model_path=f'{data_path}/models/cnn_lstm.keras'
le_dir=f'{data_path}/models/label_encoder_100.pkl'
model0 = tf.keras.models.load_model(model_path)
with open(le_dir, 'rb') as f:
    le = pickle.load(f)

In [15]:
ordered_paths=get_data_paths_ordered(data_path)

In [17]:
RANDOM_SEED = 42

# def standartize(curr_input):
#     mean = tf.math.reduce_mean(curr_input)
#     std = tf.math.reduce_std(curr_input) + 0.0001
#     return (curr_input - mean) / std

def predict_on_hdf5_with_cache_noise(test_path, model, label_encoder, noise_std, inserting_noise_p, batch_size=32):
    # Open the HDF5 file for reading
    with h5py.File(test_path, 'r') as hdf:
        # Load the features and labels
        X_test = hdf[f'vector'][:]
        y_test = label_encoder.transform((hdf[f'site_name'][:]).reshape(-1,1)).toarray()

        rng = np.random.default_rng(RANDOM_SEED)
        # noise can be negative here
        noise = rng.normal(0, noise_std, X_test.shape)
        p_of_inserting_noise = rng.binomial(1, inserting_noise_p, size=X_test.shape)
        noise = noise * p_of_inserting_noise
        X_test = X_test + noise

        # Create TensorFlow Dataset objects for efficient batch processing
        test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).map(lambda x, y: (standartize(x), y)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

        # Evaluate the model on the test dataset
        res = model.predict(test_dataset)
        return res

In [18]:
batch=256
max_epochs=20
n_incremental_samples_per_website=10

results={}
for path in tqdm(ordered_paths):
    curr_date_i=path.stem
    test_path_i = (path/'test.h5').as_posix()
    pred_i=predict_on_hdf5_with_cache_noise(test_path_i, model0, le, 0, 0, batch_size=32)
    with h5py.File(test_path_i, 'r') as hdf:
      y_testi = le.transform((hdf[f'site_name'][:]).reshape(-1,1)).toarray()
    acci=(y_testi.argmax(axis=1)==pred_i.argmax(axis=1)).mean()
    results.update({curr_date_i:acci})
    print(f'accuracy: {acci}')

  0%|          | 0/7 [00:00<?, ?it/s]

31/31 ━━━━━━━━━━━━━━━━━━━━ 6s 154ms/step


 14%|█▍        | 1/7 [00:10<01:05, 10.97s/it]

accuracy: 0.906060606060606
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 129ms/step


 29%|██▊       | 2/7 [00:15<00:36,  7.30s/it]

...

accuracy: 0.6484848484848484
31/31 ━━━━━━━━━━━━━━━━━━━━ 5s 142ms/step


 86%|████████▌ | 6/7 [00:38<00:06,  6.09s/it]

accuracy: 0.6717171717171717
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 132ms/step


100%|██████████| 7/7 [00:43<00:00,  6.28s/it]

accuracy: 0.6161616161616161


In [19]:
batch = 256
max_epochs = 20
n_incremental_samples_per_website = 10
NOISE_STDS = [0, 300, 600, 900]
NOISE_ADDING_P = [0, 0.3, 0.6, 0.9]
results = {}

for noise_std, noise_adding_p in zip(NOISE_STDS, NOISE_ADDING_P):

  results[(noise_std, noise_adding_p)] = {}
  for path in tqdm(ordered_paths):
      curr_date_i = path.stem
      test_path_i = (path/'test.h5').as_posix()
      pred_i = predict_on_hdf5_with_cache_noise(test_path_i, model0, le, noise_std, noise_adding_p, batch_size=32)
      with h5py.File(test_path_i, 'r') as hdf:
        y_testi = le.transform((hdf[f'site_name'][:]).reshape(-1,1)).toarray()
      acci = (y_testi.argmax(axis=1) == pred_i.argmax(axis=1)).mean()
      results[(noise_std, noise_adding_p)][curr_date_i] = acci
      print(f'accuracy: {acci} for date: {curr_date_i} with noise: {noise_std}')

  0%|          | 0/7 [00:00<?, ?it/s]

31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 128ms/step


 14%|█▍        | 1/7 [00:05<00:32,  5.36s/it]

accuracy: 0.906060606060606 for date: 2021_08_27 with noise: 0
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 127ms/step


 29%|██▊       | 2/7 [00:09<00:23,  4.67s/it]

...

accuracy: 0.4797979797979798 for date: 2021_10_24 with noise: 600
31/31 ━━━━━━━━━━━━━━━━━━━━ 4s 128ms/step


 71%|███████▏  | 5/7 [00:28<00:10,  5.37s/it]

accuracy: 0.5212121212121212 for date: 2021_11_28 with noise: 600
11/31 ━━━━━━━━━━━━━━━━━━━━ 2s 123ms/step

 71%|███████▏  | 5/7 [00:30<00:12,  6.03s/it]


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd


# Convert date strings to datetime objects for proper plotting
def convert_dates(date_dict):
    dates = []
    values = []
    for date_str, value in date_dict.items():
        date_obj = datetime.strptime(date_str, '%Y_%m_%d')
        dates.append(date_obj)
        values.append(float(value))
    return dates, values

# Create the plot
plt.figure(figsize=(12, 8))

# Colors for different trend lines
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728']
markers = ['o', 's', '^', 'D']

# Plot each trend line
for i, (key, values_dict) in enumerate(results.items()):
    dates, values = convert_dates(values_dict)

    # Plot the data points and line
    plt.plot(dates, values,
             color=colors[i],
             marker=markers[i],
             linewidth=2,
             markersize=8,
             label=f'Sigma Noise Level: {key[0]}, P for packet noise per sample: {key[1]}')

# Customize the plot
plt.title('Performance Trends Across Different Noise Levels On Packets Signal', fontsize=16)
plt.xlabel('Date', fontsize=12)
plt.ylabel('Test Accuracy', fontsize=12)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)

# Format x-axis dates
plt.xticks(rotation=45)
plt.tight_layout()

# Add some styling
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Show the plot
plt.show()

In [ ]:
path1 = ordered_paths[0] / 'test.h5'
with h5py.File(path1, 'r') as hdf:
  X_test = hdf[f'vector'][:]
  y_test = le.transform((hdf[f'site_name'][:]).reshape(-1,1)).toarray()

In [ ]:
X_test.shape, y_test.shape

In [ ]:
import matplotlib.pyplot as plt

plt.hist(X_test.reshape(-1))
plt.show()

In [ ]:
X_test.reshape(-1).mean(), np.median(X_test.reshape(-1))